In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import warnings
import matplotlib.colors as colors
import matplotlib.animation as animation
import matplotlib.lines as lines
import datetime
import math
%matplotlib notebook

In [6]:
warnings.filterwarnings(action='once')

!pip install sociophysicsdatahandler
from sociophysicsDataHandler import SociophysicsDataHandler

!cat auth.txt

dh = SociophysicsDataHandler()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


guest
YDPGL-XREXC-QGMZS-UAKES


C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:145: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


In [7]:



def fetchData(path):
  #Fetch and combine all data from a directory
  file_list = dh.list_files(path)
  print(dh.filelist)
  n = file_list.name.size
  print(dh.filelist.head(10))
  df_array = np.empty(n, dtype=object)

  for i in range(n):
    dh.fetch_prorail_data_from_path(path + "/" + file_list.name[i]);
    #Convert the date time column to actual date time
    dh.df['date_time_utc'] = pd.to_datetime(dh.df['date_time_utc'], unit = 'ms')
    #Swap x and y
    column_titles = {'date_time_utc':'date_time_utc', 'tracked_object':'tracked_object', 'x_pos':'y_pos', 'y_pos':'x_pos'}
    dh.df.rename(columns = column_titles, inplace = True)

    df_array[i] = dh.df

  return pd.concat(df_array)


file_path = 'ehv/platform2.1/20220404/'


# fetch the data from the first file
df = fetchData(file_path);



dh.fetch_background_image_from_path('ehv/background_images/EHV.Perron2.1_multisensor.png')



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


targeting path /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220404/
Files listed. Accessible as <this-object>.filelist
                                                 path  \
0   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
1   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
2   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
3   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
4   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
5   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
6   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
7   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
8   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
9   /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
10  /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
11  /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
12  /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
13  /storage/surfsara/ProRail_USE_LL_data/ehv/plat...   
14  /storage/su

data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220404//EHV_Platform2.1_2022040413_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220404//EHV_Platform2.1_2022040414_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220404//EHV_Platform2.1_2022040415_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220404//EHV_Platform2.1_2022040416_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220404//EHV_Platform2.1_2022040417_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220404//EHV_Platf

In [1]:


def above_line(df, line, alpha, x_tag='x_pos',y_tag='y_pos'):
    above_line_series = df[y_tag] > - math.atan(alpha/360*2*math.pi)*(df[x_tag]-line[0])+line[1]
    return above_line_series

def below_line(df, line, alpha, x_tag='x_pos',y_tag='y_pos'):
    below_line_series = df[y_tag] < - math.atan(alpha/360*2*math.pi)*(df[x_tag]-line[0])+line[1]
    return below_line_series

def left_of_line(df, line, alpha, x_tag='x_pos',y_tag='y_pos'):
    left_of_line_series = df[x_tag] < line[0]
    return left_of_line_series

df_grouped_by_id = df.groupby('tracked_object')
df_object_start_location = df_grouped_by_id.first()
df_object_end_location = df_grouped_by_id.last()

alpha = 0.8 
boarding_line_1 = [-2e4,2e3] #x, y, angle (degrees)
dx = 5e4
boarding_line_1.append(boarding_line_1[0]+dx) # [2]: x of point 2
boarding_line_1.append(boarding_line_1[1]-dx*math.tan(alpha/360*2*math.pi)) # [3]: y of point 2

boarding_line_2 = [-2e4,-0.65e4] #x, y, angle (degrees)
dx = 5e4
boarding_line_2.append(boarding_line_2[0]+dx) # [2]: x of point 2
boarding_line_2.append(boarding_line_2[1]-dx*math.tan(alpha/360*2*math.pi)) # [3]: y of point 2

boarding_line_3 = [70000,-15000,70000,10000]

boarding_line_4 = [10000,-15000,10000,10000]

#If start location is above line 1, offboarding. same for below line 2. Also to the right of the vertical line

offboarding_1 = above_line(df_object_start_location,boarding_line_1,alpha)
offboarding_2 = below_line(df_object_start_location,boarding_line_2,alpha)
offboarding_3 = (df_object_start_location['x_pos']>boarding_line_3[0])
offboarding = offboarding_1 | offboarding_2 | offboarding_3

#If end location is above line 1, onboarding. same for below line 2.

onboarding_1 = above_line(df_object_end_location,boarding_line_1,alpha)
onboarding_2 = below_line(df_object_end_location,boarding_line_2,alpha)
onboarding_3 = (df_object_end_location['x_pos']>boarding_line_3[0])
entering = left_of_line(df_object_start_location,boarding_line_4,0)
onboarding_or_entering = onboarding_1 | onboarding_2 | onboarding_3 | entering
onboarding_or_entering.head()


list_offboarding = offboarding.loc[offboarding==True].index.values
print(list_offboarding)


incoming_people_offboarding = df_object_start_location.loc[offboarding==True]
print(incoming_people_offboarding)


incoming_people_onboarding = df_object_start_location.loc[onboarding_or_entering==True]
#print(incoming_people_onboarding)

incoming_people_entering = df_object_start_location.loc[entering==True]
print(incoming_people_entering)






NameError: name 'df' is not defined